# Train data preparation

In [214]:
# Libraries
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

print('Pandas {}'.format(pd.__version__))
print('Numpy {}'.format(np.__version__))

Pandas 1.3.4
Numpy 1.20.3


In [122]:
# We get the data from link
df = pd.read_csv('https://trein.fwrite.org/AMS-Aurora-archive/2016-01/DVS_2016-01-01.csv.xz')
df.columns = ["date", "data", "code"]

In [227]:
# This function iterates through each row (day) in the given dataset
# The dataset, which in this case is one whole day is parsed as a parameter to the function
# Two lists, ritstations and trains are created at the beginning. Then in the loop, ritstation and train objects are created and added to these lists.
# Later on, the lists are combined and used in dataframe for visualization.

def GetTrainDayData(smalldf):
    # The xml data
    xml_data = smalldf["data"]
    
    # It goes through every row in the data (1 day)
    for i,train_data in enumerate(xml_data):
        root = ET.fromstring(train_data)

        # Check if there is always root
        if len(root) != 0:
            
            # This is the prefix for each element
            prefix = './/{urn:ndov:cdm:trein:reisinformatie:data:2}'
            # The data wrapper
            productDVS = root.find('{}ReisInformatieProductDVS'.format(prefix))
            # Getting the version of the data for the day (just shows how to get attributes)
            data_version = productDVS.attrib['Versie']

            # the wrapper that contain the ritstation and the train
            wrapper = root.find('{}DynamischeVertrekStaat'.format(prefix))
            
            # All data needed is extracted and put in variables here
            station_code = wrapper.find('{}StationCode'.format(prefix)).text
            station_name = wrapper.find('{}LangeNaam'.format(prefix)).text
            train_number = wrapper.find('{}TreinNummer'.format(prefix)).text
            train_type = wrapper.find('{}TreinSoort'.format(prefix)).text
            train_company = wrapper.find('{}Vervoerder'.format(prefix)).text
            train_destination = wrapper.find('{}TreinEindBestemming[@InfoStatus="Actueel"]'.format(prefix)).find('{}LangeNaam'.format(prefix)).text
            train_planned_departure = wrapper.find('{}VertrekTijd[@InfoStatus="Actueel"]'.format(prefix)).text
            train_delays = wrapper.find('{}ExacteVertrekVertraging'.format(prefix)).text
            
            rows.append({"Station Code":station_code,"Station Name":station_name,"Train Number":train_number,"Train Type":train_type,"Train Company":train_company,"Train Destination":train_destination,"Train Planned Departure":train_planned_departure,"Train Delay": train_delays})

In [234]:
from datetime import date, timedelta

rows = []
start_date = date(2016, 1, 8)
end_date = date(2016, 1, 14)
delta = timedelta(days=1)

while start_date <= end_date:
    df = pd.read_csv('https://trein.fwrite.org/AMS-Aurora-archive/2016-01/DVS_{}.csv.xz'.format(start_date.strftime("%Y-%m-%d")))
    df.columns = ["date", "data", "code"]
    GetTrainDayData(df)
    start_date += delta
    
print(len(rows))

data = pd.DataFrame(rows,columns=["Station Code","Station Name","Train Number","Train Type", "Train Company", "Train Destination", "Train Planned Departure","Train Delay"])
data.to_csv('../Datasets/test.csv')

1262482


In [ ]:
data.head(10)

In [223]:
thalys = data[data['Train Type'] == "Thalys"]
thalys['Train Destination'].value_counts()

Paris-Nord            159
Amsterdam Centraal    130
Lille Europe           11
Name: Train Destination, dtype: int64

In [222]:
companies = data['Train Type'].value_counts()
companies

Sprinter             67409
Intercity            37771
Stoptrein            23737
Intercity direct      1096
Sneltrein              983
Thalys                 300
ICE International       95
CityNightLine/IC        31
Austria Express         19
Speciale Trein          16
Name: Train Type, dtype: int64